
# Sistemas de recomendação Baseado no Collaborative Filtering
### Uma aplicação de álgebra linear

Se você já usou redes sociais, e-commerces ou serviços de streaming, provavelmente um sistema de recomendação fez parte da sua experiência mesmo que discretamente. Esses algoritmos buscam prever qual será a avaliação ou a preferência que um usuário terá sobre outra entidade do sistema, para que possa recomenda-lo algo do seu interesse.

Existem alguns tipos distintos de sistemas de recomendação, e cada um se baseia em uma forma de interação entre usuário-entidade no sistema. Por exemplo: Laura não vive sem escutar suas músicas no cotidiano. O servico de streaming que ela utiliza possui uma funcionalidade que, todo dia pega uma seleção de artistas que se fazem parte dos gêneros que Laura mais se identifica e a apresenta seus trabalhos mais recentes. Com isso, Laura está sempre por dentro das novidades. Outra forma de interação, é a que João tem com o marketplace X. Ele adora meias malucas e sempre utiliza X para comprá-las. Utilizando as avaliações de suas compras anteriores e das avaliações realizadas por outros usuários, o algoritmo da plataforma determina quais são os usuários com gostos parecidos com os de João, para assim, recomenda-lo as meias que agradaram esses usuários semelhantes.

A técnica utilizada pelo marketplace A para recomendar seus produtos aos seus usuários é conhecida como Collaboratve Filtering (CF) e é considerada uma das mais inteligentes formas de recomendação. Como narrado anteriormente, esta técnica se baseia no cálculo da similaridade entre usuários e, a partir dele, se obtém-se uma estimativa do quanto cada um pode gostar de cada item, produto ou até mesmo outros usuários.

O workflow de um sistema que utiliza CF é parecido com:

*   O usuário avalia itens na plataforma, e essas avaliações são interpretadas como uma aproximação do interesse do mesmo naquele tipo de item;
*   Atravéz do interesses dos outros usuários naquele mesmo tipo de item, o sistema mapeia quais usuários são mais similares, isto é, com interesses mais próximos com os daquele usuário;
*   O sistema recomenda itens que foram avaliados pelos usuários similares como "provável interesse" para o usuário que ainda não conhecia tais itens.

A pergunta que fica agora é: Como exatamente se calcula a similariade?
**MATEMÁTICA!** Parece meio esquisito que uma coisa aparentemente subjetiva como o de gosto que dois usuários tem em comum, possa ser expressa numa equação, mas sim, com um olhar de álgebra linear sobre esse problema, podemos obter respostas bastante precisas.

A *Similaridade de cosseno* é uma métrica poderosa para determinar o quão parecido dois objetos são, com a vantagem de não depender do seu tamanho. Nesta técnica, os dados dos objetos são tratados como vetores e dessa forma, conseguimos calcular o cosseno entre os seus ângulos. Pelo fato do cosseno variar entre -1 e 1, podemos usar esta escala para relacionar nossos objetos em um "ambiente controlado". Quanto mais próximo de 1 é o cosseno entre dois objetos, significa que eles são linearmente muito próximos, enquanto quando o cosseno vai se aproximando de -1, pode-se dizer que os objetos também são paralelos, porém tem sentidos opostos. Já quando o cosseno se aproxima de 0, diz-se que os objetos são perpendiculares, e por isso são completamente diferentes e não tem relação alguma. Sua fórmula é dada por:

<center>

$Cos(x, y) = \frac {x \cdot y}{||x|| . ||y||}$

</center>



onde, 
- $x . y$ é o produto interno entre os vetores $x$ e $y$;

- $||x||$ e $||y||$ são as normas de $x$ e $y$, ou seja, a magnitude dos vetores.



## Implementação

# Anime4u

Anime4u é um sistema de recomendação de animes que funciona da seguinte maneira: Milhares de usuários acessam a plataforma e fornecem avaliações com valores de 0 a 10 para os animes que já assistiram. Após isso, a ferramenta encontra outros usuários com gostos semelhantes e os recomenda animes que provavelmente irá gostar levando em consideração as avaliações de perfis parecidos.

In [2]:
pip install pandas Faker scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import questionary
from IPython import display
import nest_asyncio

In [4]:
# Substitua 'nome_do_arquivo.csv' pelo nome do seu arquivo CSV
df_animes = pd.read_csv('Anime_data.csv')

In [5]:
def create_fake_users(num:int) -> list[str]:
    '''
        Retorna uma lista de @num de nomes aleatórios de pessoas.
    '''
    return [Faker().name() for _ in range(num)]


def get_nth_most_popular_animes(n:int) -> list[int]:
    animes =df_animes[(df_animes["Title"] != "None")&(df_animes["Type"] == "TV") &( ~ df_animes['Genre'].str.contains("Kids",na=False)) & ( ~ df_animes['Genre'].str.contains("Hentai",na=False)) ]
    most_popular_animes = animes.nlargest(n,"ScoredBy")
    return most_popular_animes.index

def get_rand_animes(max:int) -> list[str]:
    '''
        Retorna uma lista de (no máximo) @max de nomes de animes aleatórios + alguns animes populares

        Meta: Garantir que 1/3 dos animes estejam entre os mais populares.
    '''
    num_animes = random.randint(1,max)
    most_popular_animes = get_nth_most_popular_animes(10)
    rand_anime_indexes = {random.randint(0,len(most_popular_animes)) for i in range(0,num_animes)}
    animes = [df_animes["Title"][i] for i in rand_anime_indexes]

    return animes

In [6]:
def rand_rates(num_users, num_max_animes):
    rates = {}

    for u in create_fake_users(num_users):
        user = {}
        for a in get_rand_animes(num_max_animes):
            rate = round(random.uniform(1, 10), 1)
            user[a] = rate

        rates[u] = user
    
    return rates

rates = rand_rates(100, 5)
df_rates = pd.DataFrame(rates)

print(df_rates, df_rates.info)


                                 James Rogers  William Aguilar  Molly Pollard  \
Cowboy Bebop                              4.6              NaN            9.5   
Naruto                                    7.5              NaN            NaN   
Hungry Heart: Wild Striker                8.3              NaN            NaN   
Initial D Fourth Stage                    NaN              9.6            4.6   
Trigun                                    NaN              NaN            NaN   
Bouken Ou Beet                            NaN              NaN            NaN   
Cowboy Bebop: Tengoku no Tobira           NaN              NaN            NaN   
Monster                                   NaN              NaN            NaN   
Eyeshield 21                              NaN              NaN            NaN   
Hachimitsu to Clover                      NaN              NaN            NaN   
Witch Hunter Robin                        NaN              NaN            NaN   

                           

In [7]:
def cos_between_vectors(A, B):
    # produto interno u e v / |v| * |u|
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)

    if norm_a * norm_b == 0:
        return 0
    
    return A@B / (norm_a * norm_b)

In [8]:
matriz_avaliacoes = df_rates.to_numpy()
# Colunas - Usuarios | Linhas = Anime
matriz_avaliacoes = np.nan_to_num(matriz_avaliacoes)
print(matriz_avaliacoes, "\n\n")
print(matriz_avaliacoes[:,4])

print(cos_between_vectors(matriz_avaliacoes[:, 1], matriz_avaliacoes[:, 2]))

[[4.6 0.  9.5 ... 0.  0.  0. ]
 [7.5 0.  0.  ... 0.  3.  0. ]
 [8.3 0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  6.5 3.4]
 [0.  0.  0.  ... 0.  7.8 5.6]
 [0.  0.  0.  ... 0.  5.6 0. ]] 


[0.  0.  0.  0.  1.9 0.  6.3 0.  0.  0.  0. ]
0.43580854827079


In [9]:
def print_animes(num):
    anime_titles = get_animes_titles(get_nth_most_popular_animes(num)) 
    for index,item in enumerate(anime_titles):
        if(index != len(anime_titles)):
            print(index," - ",item)


def get_chosen_anime_indexes(num:int,text:str) -> list[int]: 
    print_animes(num)
    anime_input = input(text)
    chosen_items = anime_input.split(" ")
    chosen_items = list(map(int, chosen_items))

    anime_indexes = [get_nth_most_popular_animes(num)[i] for i in chosen_items]

    return anime_indexes

def get_animes_titles(anime_indexes):
    return [df_animes["Title"][i] for i in anime_indexes]

chosen_indexes = get_chosen_anime_indexes(10,"Escolha o número seus animes. Separe por espaço.")
chosen_titles = get_animes_titles(chosen_indexes)


0  -  Death Note
1  -  Shingeki no Kyojin
2  -  Sword Art Online
3  -  Fullmetal Alchemist: Brotherhood
4  -  One Punch Man
5  -  Tokyo Ghoul
6  -  Naruto
7  -  Angel Beats!
8  -  Code Geass: Hangyaku no Lelouch
9  -  No Game No Life


ValueError: invalid literal for int() with base 10: ''

In [ ]:
# [print(i) for i in df_rates.columns]
df = pd.DataFrame(df_rates.pivot(columns=[print(i) for i in df_rates.columns]))
# pd.plotting.scatter_matrix(df, alpha = 0.2, figsize = (6, 6), diagonal = 'kde')

Anne Mcclain
Courtney Brown
Natasha Kim
Olivia Harper
Stephanie Rice
Derrick Gray
Erin Henderson
Juan Miller
Daniel Rogers
Ronald Escobar
Socrates
Alberto
Cleber


KeyError: 'None of [None, None, None, None, None, None, None, None, None, None, None, None, None] are in the columns'

In [12]:

def get_new_rate():
    user_rates = {}
    chosen_indexes = get_chosen_anime_indexes(10,"Escolha o número seus animes. Separe por espaço.")
    chosen_titles = get_animes_titles(chosen_indexes)
    for a in chosen_titles:
        rate = input(f"Dê sua nota para {a}:")
        user_rates[a] = round(float(rate),1)
    user_name = input("Informe seu nome")

    return user_name, user_rates


def save_new_rate(user_name, user_rates, rates):
    rates[user_name] = user_rates
    return rates


def get_similars_users(user, users_matrix):  # caso nao exista ngm, recomendar os animes mais populares
    index_of_similars = []

    # Itera pelas colunas da matriz (usuarios) calculando o cosseno e
    for c in range(users_matrix.shape[1] - 1):
        cos = cos_between_vectors(users_matrix[:, c], user)
        if cos > 0.5 :
            index_of_similars.append(c)
        # print(c, cos)

    return index_of_similars

def get_recoms(df_rates, matrix_rates):
    # retornar os nomes dos animes
    # quais animes? aqueles que foram avaliados por usuarios semelhantes ao novo usuario.

    all_users_names = [i for i in df_rates.columns]
    
    return
    
get_recoms(df_rates, matriz_avaliacoes)

In [ ]:


u_name, u_rates = get_new_rate()

rates = save_new_rate(u_name, u_rates, rates)

# DataFrame rates
df_rates = pd.DataFrame(rates)



def get_animes_from_user(user):
    return df_rates[df_rates[user]][user]

def get_evaluated_anime_by_one_user_but_not_other(name_user1,name_user2,df):
    # animes_user1 = get_animes_from_user(name_user1)
    # animes_user2 = get_animes_from_user(name_user2)
    animes_in_user_1_but_not_user_2 = df[((df[name_user1] > 1.0) )& (df[name_user2] == 0)][name_user1]
    return animes_in_user_1_but_not_user_2


def get_users_with_similar_rates_but_missing_values(matrix,name):
    users = get_similars_users(matrix[:,-1],matrix)
    all_users_names = [i for i in df_rates.columns]

    print("Usuarios mais similares: ")

    for i in users:
        print(all_users_names[i],":")
        print("Animes Recomendados:")
        evaluations = get_evaluated_anime_by_one_user_but_not_other(all_users_names[i],u_name,df_rates)
        if(evaluations.empty == False):
            print(evaluations)




# numPy Matrix rates

# print(df_rates)

df_rates = df_rates.fillna(0)

matrix_rates = df_rates.to_numpy()


get_users_with_similar_rates_but_missing_values(matrix_rates,u_name)
# print("\n\nMatriz\n\n",matrix_rates,"\n\n")
# Recupera os usuários parecidos com o novo usuário
# users = get_similars_users(matrix_rates[:,-1],matrix_rates)
# all_users_names = [i for i in df_rates.columns]
# # print(df_rates)
# print("Most Similar Users: ",[all_users_names[i] for i in users])
# print(get_evaluated_anime_by_one_user_but_not_other("Alberto","Socrates",df_rates))
# print(newrates[newrates[all_users_names[6]] != 0]["Socrates"])


0  -  Death Note
1  -  Shingeki no Kyojin
2  -  Sword Art Online
3  -  Fullmetal Alchemist: Brotherhood
4  -  One Punch Man
5  -  Tokyo Ghoul
6  -  Naruto
7  -  Angel Beats!
8  -  Code Geass: Hangyaku no Lelouch
9  -  No Game No Life
Usuarios mais similares: 
Marcos :
Animes Recomendados:
Death Note          8.0
Sword Art Online    2.0
Name: Marcos, dtype: float64
Kavalo :
Animes Recomendados:
Bruno :
Animes Recomendados:
Leo :
Animes Recomendados:
Clara :
Animes Recomendados:
Ope :
Animes Recomendados:
LLLA :
Animes Recomendados:
AMAKD :
Animes Recomendados:


## Execução:

In [13]:


u_name, u_rates = get_new_rate()

rates = save_new_rate(u_name, u_rates, rates)

# DataFrame rates
df_rates = pd.DataFrame(rates)


df_rates = df_rates.fillna(0)

matrix_rates = df_rates.to_numpy()


get_users_with_similar_rates_but_missing_values(matrix_rates,u_name)

0  -  Death Note
1  -  Shingeki no Kyojin
2  -  Sword Art Online
3  -  Fullmetal Alchemist: Brotherhood
4  -  One Punch Man
5  -  Tokyo Ghoul
6  -  Naruto
7  -  Angel Beats!
8  -  Code Geass: Hangyaku no Lelouch
9  -  No Game No Life


ValueError: invalid literal for int() with base 10: ''